<a href="https://colab.research.google.com/github/YuichiIkeda600422/colab_sample/blob/main/AutoGluon_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --upgrade "mxnet<2.0.0"
!pip install autogluon

     |████████████████████████████████| 1.5MB 8.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 884 kB 8.8 MB/s 
     |████████████████████████████████| 136 kB 25.1 MB/s 
     |████████████████████████████████| 127 kB 26.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [ ]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = train_data.head(10000)
print(train_data.head())

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073


   age   workclass  fnlwgt  ... hours-per-week  native-country   class
0   25     Private  178478  ...             40   United-States   <=50K
1   23   State-gov   61743  ...             35   United-States   <=50K
2   46     Private  376789  ...             15   United-States   <=50K
3   55           ?  200235  ...             50   United-States    >50K
4   36     Private  224541  ...             40     El-Salvador   <=50K

[5 rows x 15 columns]


In [ ]:
label_column = 'class'
print("Summary of class variable: \n", train_data[label_column].describe())

NumExpr defaulting to 2 threads.


Summary of class variable: 
 count      10000
unique         2
top        <=50K
freq        7568
Name: class, dtype: object


In [ ]:
%%time
dir = 'agModels-predictClass' # specifies folder where to store trained models
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Beginning AutoGluon training ...
AutoGluon will save models to agModels-predictClass/
AutoGluon Version:  0.0.15
Train Data Rows:    10000
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12686.97 MB
	Train Data (Original)  Memory Usage: 5.87 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...


CPU times: user 1min 30s, sys: 3.15 s, total: 1min 33s
Wall time: 1min 3s


In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                         model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      weighted_ensemble_k0_l1      0.854       0.346473  10.416801                0.002270           0.606055            1       True         12
1   RandomForestClassifierEntr      0.852       0.110892   2.739066                0.110892           2.739066            0       True          2
2           LightGBMClassifier      0.850       0.018725   0.498305                0.018725           0.498305            0       True          7
3           CatboostClassifier      0.850       0.022314   5.249071                0.022314           5.249071            0       True          9
4     LightGBMClassifierCustom      0.847       0.024668   0.923192                0.024668           0.923192            0       True         11
5         LightGBMClassifierXT      0.846       0.018635   0.5

In [ ]:
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(labels=[label_column],axis=1) # delete label column to prove we're not cheating
print(test_data_nolab.head())

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


   age          workclass  fnlwgt  ... capital-loss  hours-per-week  native-country
0   31            Private  169085  ...            0              20   United-States
1   17   Self-emp-not-inc  226203  ...            0              45   United-States
2   47            Private   54260  ...         1887              60   United-States
3   21            Private  176262  ...            0              30   United-States
4   17            Private  241185  ...            0              20   United-States

[5 rows x 14 columns]


In [ ]:
predictor = task.load(dir) # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


Evaluation: accuracy on test data: 0.8556658818712253
Evaluations on test data:
{
    "accuracy": 0.8556658818712253,
    "accuracy_score": 0.8556658818712253,
    "balanced_accuracy_score": 0.7710491402616739,
    "matthews_corrcoef": 0.5803423291109661,
    "f1_score": 0.8556658818712253
}


Predictions:   [' <=50K' ' <=50K' ' >50K' ... ' >50K' ' <=50K' ' <=50K']


Detailed (per-class) classification report:
{
    " <=50K": {
        "precision": 0.8848260924958593,
        "recall": 0.9320896523956516,
        "f1-score": 0.907843137254902,
        "support": 7451
    },
    " >50K": {
        "precision": 0.7364583333333333,
        "recall": 0.6100086281276963,
        "f1-score": 0.6672958942897593,
        "support": 2318
    },
    "accuracy": 0.8556658818712253,
    "macro avg": {
        "precision": 0.8106422129145963,
        "recall": 0.7710491402616739,
        "f1-score": 0.7875695157723306,
        "support": 9769
    },
    "weighted avg": {
        "precision": 0.8496212132104939,
        "recall": 0.8556658818712253,
        "f1-score": 0.8507657998413284,
        "support": 9769
    }
}
